In [9]:
from pathlib import Path
import pandas as pd
import numpy as np

from Bio.PDB import PDBParser, PDBIO, Select

In [2]:
predictions = pd.read_csv("../data/AlphaFold-disorder/UP000005640_9606_HUMAN_v3_pred.tsv", sep="\t")
grouped_predictions = predictions.groupby("name")

structures = Path("../data/AlphaFoldDB/UP000005640_9606_HUMAN_v3/").rglob("*.pdb")


In [42]:
example = "AF-A4D2P6-F1-model_v3"

df = grouped_predictions.get_group(example)

# forward and reverse rolling 
wdw_fdw = df["disorder-25"].rolling(25)
wdw_rev = df["disorder-25"][::-1].rolling(25)

ordered_positions = []

for wdw in [wdw_fdw, wdw_rev]:
    ordered = df[wdw.min() <= 0.581]
    ordered_positions.extend(ordered["pos"].values)
ordered_positions = set(ordered_positions)

/tmp/ipykernel_354298/2902748076.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ordered = df[wdw.min() <= 0.581]


In [39]:
parser = PDBParser()
struc = parser.get_structure(example, f"../data/AlphaFoldDB/UP000005640_9606_HUMAN_v3/{example}.pdb")

In [43]:
io = PDBIO()

class ResSelect(Select):
    def accept_residue(self, res):
        if res.id[1] in ordered_positions:
            return True
        else:
            return False


io = PDBIO()
io.set_structure(struc)
io.save('../data/example_trimmed.pdb', ResSelect(), preserve_atom_numbering=True)